In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('state_legislative_election_results_1971_2012.csv')
df.head()

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,State,District,Year,Party,Incumbent,Dem Votes,GOP Votes,Other Votes
0,AK,1,1992,D,0,3019,3002,0
1,AK,2,1992,D,1,4704,2169,0
2,AK,3,1992,D,1,5210,2233,0
3,AK,4,1992,R,1,1754,4538,294
4,AK,5,1992,D,1,3953,0,1153


In [7]:
df['Efficiency Gap'] = 0
df.head()

,State,District,Year,Party,Incumbent,Dem Votes,GOP Votes,Other Votes,Efficiency Gap
0,AK,1,1992,D,0,3019,3002,0,0
1,AK,2,1992,D,1,4704,2169,0,0
2,AK,3,1992,D,1,5210,2233,0,0
3,AK,4,1992,R,1,1754,4538,294,0
4,AK,5,1992,D,1,3953,0,1153,0


In [20]:
earliest_years = []
states = []
for i in range(len(df)):
    state = df['State'][i]
    if state not in states:
        states.append(state)
        earliest_years.append(df['Year'][i])
print("Earliest years of", len(earliest_years), 'states:', earliest_years)
print()
print('Earliest common year:', max(earliest_years))

Earliest years of 38 states: [1992, 1974, 1982, 1972, 1972, 1972, 1972, 1982, 1992, 1982, 1972, 1982, 1992, 1972, 1971, 1978, 1978, 1972, 1972, 1972, 1979, 1974, 2002, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1972, 1974, 1972, 1976, 1972, 1985, 1972, 1992]

Earliest common year: 2002


### Let's decide not to cut data
yum!

In [26]:
def calculateEG(d, r, winner):
    if winner == 'D':
        return (d - (d + r) / 2 - r) / (d + r)
    if winner == 'R':
        return (r - (d + r) / 2 - d) / (d + r)
    else:
        return 0


for i in range(len(df)):
    df.loc[i, 'Efficiency Gap'] = calculateEG(df['Dem Votes'][i], df['GOP Votes'][i], df['Party'][i])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
df

,State,District,Year,Party,Incumbent,Dem Votes,GOP Votes,Other Votes,Efficiency Gap
0,AK,1,1992,D,0,3019,3002,0,-0.497177
1,AK,2,1992,D,1,4704,2169,0,-0.131165
2,AK,3,1992,D,1,5210,2233,0,-0.100027
3,AK,4,1992,R,1,1754,4538,294,-0.057533
4,AK,5,1992,D,1,3953,0,1153,0.500000
5,AK,6,1992,D,1,3802,0,0,0.500000
6,AK,7,1992,R,1,2433,3078,482,-0.382961
7,AK,8,1992,R,0,2438,2634,524,-0.461356
8,AK,9,1992,D,1,2561,2343,672,-0.455546
9,AK,10,1992,R,0,2092,3607,899,-0.234164
